In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math
from datetime import datetime, timedelta
from time import strftime
import os
from requests.adapters import HTTPAdapter, Retry
import json
import pandas as pd
import glob

from pbp_utils import get_matchup_games, retrieve_schedule, get_livedata_from_game, scrape_month_playbyplay
from team_utils import get_team_info, save_team_info_to_file


In [2]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2024-11-10'

In [3]:
with open('data/team_info.json', 'r') as f:
    team_info = json.load(f)

In [4]:
# doesn't include international games
start_2024 = '2024-10-08'
so_far = (retrieve_schedule(start_2024, yesterday))
# so_far

In [5]:
first_week = retrieve_schedule(start_2024, '2024-10-14')
psg = get_livedata_from_game(first_week)

psg_df = pd.DataFrame(psg)

In [6]:
def clean_pbp_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Renames columns by removing the 'details_' prefix and performs other cleaning operations.

    Parameters:
        df (pd.DataFrame): The original DataFrame with columns to clean.

    Returns:
        pd.DataFrame: A DataFrame with cleaned columns.
    """
    # Rename columns by removing the 'details_' prefix
    df = df.rename(columns=lambda x: x.replace('details_', '') if x.startswith('details_') else x)

    # List of columns to drop if they exist
    columns_to_drop = ['maxRegulationPeriods', 'timeInPeriod', 'period_type', 'highlightClip',
                       'highlightClipFr', 'discreteClip', 'discreteClipFr', 'highlightClipSharingUrl','highlightClipSharingUrlFr']
    
    # Drop columns that exist in the DataFrame
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    # Rename columns
    column_rename_map = {
        'period_number': 'period', 'timeRemaining': 'time',
        'homeTeamDefendingSide': 'homeSide', 'typeCode': 'code',
        'typeDescKey': 'desc', 'eventOwnerTeamId': 'team', 'losingPlayerId': 'loser',
        'winningPlayerId': 'winner', 'xCoord': 'x', 'yCoord': 'y',
        'zoneCode': 'zone', 'shootingPlayerId': 'shooter', 'goalieInNetId': 'goalie',
        'hittingPlayerId': 'hitter', 'hitteePlayerId': 'hittee',
        'drawnByPlayerId': 'drawnBy', 'servedByPlayerId': 'servedBy',
        'committedByPlayerId': 'committedBy', 'blockingPlayerId': 'blocker',
        'playerId': 'player', 'scoringPlayerId': 'scorer', 'assist1PlayerId': 'assist1',
        'assist2PlayerId': 'assist2'
    }
    df = df.rename(columns={k: v for k, v in column_rename_map.items() if k in df.columns})

    # Convert player_id related columns to integers
    player_id_columns = ['player','loser', 'winner', 'hitter', 'hittee', 'shooter', 'goalie', 
                         'blocker', 'playerId', 'scorer', 'assist1',
                         'assist2', 'committedBy', 'drawnBy', 'servedBy']

    for col in player_id_columns:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)

    return df

In [7]:
psg_df = clean_pbp_data(psg_df)

In [8]:
psg_df.head(10)

,gid,eventId,sortOrder,period,time,situationCode,homeSide,code,desc,team,loser,winner,x,y,zone,hitter,hittee,shotType,shooter,goalie,awaySOG,homeSOG,reason,blocker,player,secondaryReason,scorer,scoringPlayerTotal,assist1,assist1PlayerTotal,awayScore,homeScore,assist2,assist2PlayerTotal,code,descKey,duration,committedBy,drawnBy,servedBy
0,2024020003,9,8,1,20:00,1551,left,520,period-start,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
1,2024020003,8,11,1,20:00,1551,left,502,faceoff,19.0,8482665,8480023,0.0,0.0,N,0,0,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
2,2024020003,75,12,1,19:52,1551,left,503,hit,55.0,0,0,93.0,28.0,D,8477955,8476892,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
3,2024020003,78,13,1,19:50,1551,left,503,hit,19.0,0,0,67.0,-40.0,N,8482077,8474586,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
4,2024020003,61,22,1,19:22,1551,left,506,shot-on-goal,19.0,0,0,-61.0,27.0,O,0,0,wrist,8477402,8475831,1.0,0.0,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
5,2024020003,64,25,1,19:04,1551,left,507,missed-shot,19.0,0,0,-61.0,-7.0,O,0,0,wrist,8479385,8475831,NaN,NaN,hit-crossbar,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
6,2024020003,70,30,1,18:49,1551,left,506,shot-on-goal,55.0,0,0,79.0,-30.0,O,0,0,wrist,8477416,8476412,1.0,1.0,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
7,2024020003,76,35,1,18:29,1551,left,507,missed-shot,55.0,0,0,83.0,9.0,O,0,0,backhand,8480009,8476412,NaN,NaN,wide-right,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
8,2024020003,96,36,1,18:26,1551,left,503,hit,19.0,0,0,91.0,-26.0,N,8475170,8478840,NaN,0,0,NaN,NaN,NaN,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0
9,2024020003,77,37,1,18:13,1551,left,508,blocked-shot,55.0,0,0,73.0,-9.0,D,0,0,NaN,8478840,0,NaN,NaN,blocked,8470600,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0,0,0


In [9]:
# db_config = {
#     'dbname': 'nhl_players_db',
#     'user': 'postgres',
#     'password': 'willIam21278$',
#     'host': 'localhost',
#     'port': '5432'
# }

In [18]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Construct the database configuration dictionary
db_config = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

In [19]:
from insert_player import insert_player

In [20]:
def insert_players(player_ids: list, db_config: dict) -> None:
    """
    Processes an array of player IDs and inserts each into the nhl_players_db PostgreSQL database.

    Parameters:
        player_ids (list): List of player IDs to be inserted.
        db_config (dict): Database configuration with keys: dbname, user, password, host, port.
    """
    for player_id in player_ids:
        insert_player(player_id, db_config)

insert_players(psg_df['shooter'].unique(), db_config)

HTTP error occurred while fetching player data: 404 Client Error: Not Found for url: https://api-web.nhle.com/v1/player/0/landing
Player Data Types and Values:
player_id: int - 8477402
first_name: str - Pavel
last_name: str - Buchnevich
full_name: str - Pavel Buchnevich
position: str - L
jersey_number: int - 89
date_of_birth: str - 1995-04-17
nationality: str - RUS
height: int - 73
weight: int - 196
shoots: str - L
current_team_id: int - 19
current_team_name: str - St. Louis Blues
current_team_abbreviation: str - STL
is_active: bool - True
Player Pavel Buchnevich inserted/updated successfully.
Player Data Types and Values:
player_id: int - 8479385
first_name: str - Jordan
last_name: str - Kyrou
full_name: str - Jordan Kyrou
position: str - C
jersey_number: int - 25
date_of_birth: str - 1998-05-05
nationality: str - CAN
height: int - 73
weight: int - 196
shoots: str - R
current_team_id: int - 19
current_team_name: str - St. Louis Blues
current_team_abbreviation: str - STL
is_active: boo

In [8]:
def save_pbp_month(month: int, year: int, directory: str = 'data/') -> None:
    """
    Scrapes play-by-play data for a given month and year, cleans it, and saves it to a CSV file
    with the filename format 'nhl_pbp_{month}_{year}.csv'.
    
    This function integrates the scraping and saving processes by calling the `scrape_month_playbyplay`
    function internally.
    
    Parameters:
        month (int): The month number (1-12).
        year (int): The four-digit year (e.g., 2023).
        directory (str): The directory where the file will be saved. Defaults to 'data/'.
    """
    import os
    
    # Scrape the play-by-play data for the specified month and year
    df = scrape_month_playbyplay(year, month)
    
    # Clean the scraped data
    df_cleaned = clean_pbp_data(df)
    
    # Format the filename with leading zeros for month and last two digits of year
    filename = f'nhl_pbp_{month:02d}_{str(year)[-2:]}.csv'
    file_path = os.path.join(directory, filename)
    
    # Ensure the target directory exists
    os.makedirs(directory, exist_ok=True)
    
    # Save the cleaned DataFrame to a CSV file
    df_cleaned.to_csv(file_path, index=False)
    
    print(f"Play-by-play data for {month}/{year} saved to {file_path}")

In [9]:
# Assuming you have the `scrape_month_playbyplay` and `clean_pbp_data` functions defined,
# you can save the play-by-play data for October 2023 as follows:

# save_pbp_month(month=10, year=2021)
# save_pbp_month(month=11, year=2021)
# save_pbp_month(month=12, year=2021)
# save_pbp_month(month=1, year=2022)
# save_pbp_month(month=2, year=2022)
# save_pbp_month(month=3, year=2022)
# save_pbp_month(month=4, year=2022)
# save_pbp_month(month=5, year=2022)

In [10]:
def load_pbp_files(directory='data', prefix='nhl_pbp_'):
    """
    Loads all play-by-play (PBP) CSV files in the specified directory that start with the given prefix.

    Parameters:
        directory (str): The path to the directory containing PBP files. Defaults to 'data'.
        prefix (str): The prefix that PBP files start with. Defaults to 'nhl_pbp_'.

    Returns:
        pd.DataFrame: A concatenated DataFrame containing data from all matching PBP files.
    """
    # Construct the file pattern
    pattern = os.path.join(directory, f'{prefix}*.csv')
    
    # Retrieve all file paths matching the pattern
    file_paths = glob.glob(pattern)
    
    # List to hold individual DataFrames
    df_list = []
    
    # Iterate over each file and read its contents
    for file in file_paths:
        try:
            df = pd.read_csv(file)
            df_list.append(df)
            print(f"Successfully loaded: {file}")
        except Exception as e:
            print(f"Error loading {file}: {e}")
    
    # Concatenate all DataFrames if any are loaded
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
        print(f"Total files loaded and concatenated: {len(df_list)}")
        return combined_df
    else:
        print("No files found matching the pattern.")
        return pd.DataFrame()

In [11]:
df = load_pbp_files()
df.head(50)

Successfully loaded: data/nhl_pbp_03_22.csv
Successfully loaded: data/nhl_pbp_11_22.csv
Successfully loaded: data/nhl_pbp_11_23.csv
Successfully loaded: data/nhl_pbp_03_23.csv
Successfully loaded: data/nhl_pbp_11_21.csv
Successfully loaded: data/nhl_pbp_01_24.csv
Successfully loaded: data/nhl_pbp_03_24.csv
Successfully loaded: data/nhl_pbp_01_22.csv
Successfully loaded: data/nhl_pbp_01_23.csv
Successfully loaded: data/nhl_pbp_05_23.csv
Successfully loaded: data/nhl_pbp_05_22.csv
Successfully loaded: data/nhl_pbp_05_24.csv
Successfully loaded: data/nhl_pbp_10_21.csv
Successfully loaded: data/nhl_pbp_10_23.csv
Successfully loaded: data/nhl_pbp_02_23.csv
Successfully loaded: data/nhl_pbp_02_22.csv
Successfully loaded: data/nhl_pbp_10_22.csv
Successfully loaded: data/nhl_pbp_12_23.csv
Successfully loaded: data/nhl_pbp_12_22.csv
Successfully loaded: data/nhl_pbp_12_21.csv
Successfully loaded: data/nhl_pbp_02_24.csv
Successfully loaded: data/nhl_pbp_04_22.csv
Successfully loaded: data/nhl_pb

,gid,eventId,sortOrder,period,time,situationCode,homeSide,code,desc,team,loser,winner,x,y,zone,reason,hitter,hittee,player,shotType,shooter,goalie,scorer,scoringPlayerTotal,assist1,assist1PlayerTotal,assist2,assist2PlayerTotal,awayScore,homeScore,blocker,awaySOG,homeSOG,secondaryReason,code.1,descKey,duration,committedBy,drawnBy,servedBy
0,2021020852,51,8,1,20:00,1551.0,left,520,period-start,NaN,0,0,NaN,NaN,NaN,NaN,0,0,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,2021020852,52,9,1,20:00,1551.0,left,502,faceoff,9.0,8478010,8478400,0.0,0.0,N,NaN,0,0,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,2021020852,8,10,1,19:45,1551.0,left,516,stoppage,NaN,0,0,NaN,NaN,NaN,offside,0,0,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,2021020852,53,11,1,19:45,1551.0,left,502,faceoff,9.0,8478010,8478400,20.0,22.0,N,NaN,0,0,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,2021020852,9,12,1,19:31,1551.0,left,503,hit,9.0,0,0,-89.0,33.0,O,NaN,8477482,8476292,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
5,2021020852,10,13,1,19:24,1551.0,left,503,hit,9.0,0,0,-85.0,-36.0,O,NaN,8480801,8478416,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
6,2021020852,11,15,1,19:17,1551.0,left,503,hit,9.0,0,0,-88.0,34.0,O,NaN,8477482,8476292,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
7,2021020852,12,18,1,19:04,1551.0,left,525,takeaway,9.0,0,0,-71.0,35.0,O,NaN,0,0,8477015,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
8,2021020852,54,19,1,18:56,1551.0,left,507,missed-shot,9.0,0,0,-43.0,22.0,O,wide-of-net,0,0,0,wrist,8479458,8476883,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
9,2021020852,55,20,1,18:49,1551.0,left,505,goal,9.0,0,0,-72.0,-7.0,O,NaN,0,0,0,tip-in,0,8476883,8474589,6.0,8478469,23.0,8477015,22.0,1.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [12]:
# Filter for shot-on-goal and goal events, along with their preceding events
all_events = df.copy()

# Add a column to indicate if the **current** event is a shot or goal
all_events['is_shot_or_goal'] = all_events['desc'].isin(['shot-on-goal', 'goal'])

# Add a column to indicate if the **next** event (since data is descending) is a shot or goal
all_events['next_is_shot_or_goal'] = all_events['is_shot_or_goal'].shift(1)

# Filter for shot-on-goal and goal events, and events immediately preceding them
shot_goal_events = all_events[
    all_events['is_shot_or_goal'] | all_events['next_is_shot_or_goal']
].copy()

# Add a column to easily identify which rows are shots/goals and which are preceding events
shot_goal_events['event_type'] = np.where(
    shot_goal_events['is_shot_or_goal'],
    'shot_or_goal',
    'preceding_event'
)

# Sort the filtered data to maintain the original descending order
shot_goal_events.sort_index(inplace=True)

In [13]:
shot_goal_events.head(15)

,gid,eventId,sortOrder,period,time,situationCode,homeSide,code,desc,team,loser,winner,x,y,zone,reason,hitter,hittee,player,shotType,shooter,goalie,scorer,scoringPlayerTotal,assist1,assist1PlayerTotal,assist2,assist2PlayerTotal,awayScore,homeScore,blocker,awaySOG,homeSOG,secondaryReason,code.1,descKey,duration,committedBy,drawnBy,servedBy,is_shot_or_goal,next_is_shot_or_goal,event_type
9,2021020852,55,20,1,18:49,1551.0,left,505,goal,9.0,0,0,-72.0,-7.0,O,NaN,0,0,0,tip-in,0,8476883,8474589,6.0,8478469,23.0,8477015,22.0,1.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
10,2021020852,56,23,1,18:49,1551.0,left,502,faceoff,14.0,8482116,8474564,0.0,0.0,N,NaN,0,0,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
22,2021020852,62,45,1,16:17,1551.0,left,505,goal,9.0,0,0,-82.0,23.0,O,NaN,0,0,0,wrist,0,8476883,8477482,9.0,8478400,1.0,8480801,21.0,2.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
23,2021020852,63,48,1,16:17,1551.0,left,502,faceoff,9.0,8474564,8477426,0.0,0.0,N,NaN,0,0,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
25,2021020852,64,51,1,15:40,1551.0,left,506,shot-on-goal,9.0,0,0,-69.0,29.0,O,NaN,0,0,0,snap,8477015,8476883,0,NaN,0,NaN,0,NaN,NaN,NaN,0,1.0,0.0,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
26,2021020852,20,52,1,15:40,1551.0,left,516,stoppage,NaN,0,0,NaN,NaN,NaN,goalie-stopped-after-sog,0,0,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
28,2021020852,66,56,1,15:13,1551.0,left,506,shot-on-goal,14.0,0,0,55.0,19.0,O,NaN,0,0,0,slap,8475167,8476899,0,NaN,0,NaN,0,NaN,NaN,NaN,0,1.0,1.0,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal
29,2021020852,67,62,1,14:33,1551.0,left,506,shot-on-goal,9.0,0,0,-78.0,21.0,O,NaN,0,0,0,wrist,8475766,8476883,0,NaN,0,NaN,0,NaN,NaN,NaN,0,2.0,1.0,NaN,NaN,NaN,NaN,0,0,0,True,True,shot_or_goal
30,2021020852,21,63,1,14:31,1551.0,left,503,hit,14.0,0,0,-57.0,-39.0,D,NaN,8470621,8479580,0,NaN,0,0,0,NaN,0,NaN,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,False,True,preceding_event
33,2021020852,68,68,1,13:53,1551.0,left,506,shot-on-goal,9.0,0,0,-57.0,-19.0,O,NaN,0,0,0,wrist,8480801,8476883,0,NaN,0,NaN,0,NaN,NaN,NaN,0,3.0,1.0,NaN,NaN,NaN,NaN,0,0,0,True,False,shot_or_goal


In [14]:
def get_player_info(player_id):
    """
    Description:
        Retrieve information for a specific player from the NHL API and save it as a JSON file.

    Args:
        player_id (int): The unique identifier for the player.

    Returns:
        None

    Saves:
        A JSON file containing the player's information in the 'data/' directory.
        The file is named using the format 'player_{player_id}.json'.

    Example:
        get_player_info(8478236)
        # This will save the player's information to 'data/player_8478236.json'
    """

    # Construct the URL to fetch player information
    url = f'https://api-web.nhle.com/v1/player/{player_id}/landing'

    try:
        # Send a GET request to the API
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes

        # Parse the JSON response
        player_data = response.json()

        # Define the directory and file path
        directory = 'data'
        file_name = f'player_{player_id}.json'
        file_path = os.path.join(directory, file_name)

        # Ensure the directory exists
        os.makedirs(directory, exist_ok=True)

        # Save the JSON data to a file
        with open(file_path, 'w') as json_file:
            json.dump(player_data, json_file, indent=4)

        print(f"Player information saved to {file_path}")

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # Handle HTTP errors
    except Exception as err:
        print(f"An error occurred: {err}")  # Handle other possible errors

In [30]:
get_player_info(player_id=8475193)

Player information saved to data\player_8475193.json


In [141]:
import math

# Define goal location coordinates to calculate distance for shot
GOAL_LOCATION = [89, 0]

# Modify shot location coordinates
# If x coord is negative, reflect y coord so side which shot came from is consistent
df['y'] = np.where(df['x'] < 0, df['y'] * -1, df['y'])
df['x'] = np.where(df['x'] < 0, np.absolute(df['x']), df['x'])

def calculate_shot_distance(x, y):
    dist = math.dist([x, y], GOAL_LOCATION)
    return round(dist, 2)

# Calculate shot distance for shots on goal
df['shotDist'] = df.apply(lambda row: calculate_shot_distance(row['x'], row['y']) 
                          if row['desc'] == 'shot-on-goal' else None, axis=1)

# Display the first few rows to verify the new column
df[['desc', 'x', 'y', 'shotDist']].head(10)


,desc,x,y,shotDist
0,period-start,NaN,NaN,NaN
1,faceoff,0.0,0.0,NaN
2,hit,93.0,28.0,NaN
3,hit,67.0,-40.0,NaN
4,shot-on-goal,61.0,-27.0,38.90
5,missed-shot,61.0,7.0,NaN
6,shot-on-goal,79.0,-30.0,31.62
7,missed-shot,83.0,9.0,NaN
8,hit,91.0,-26.0,NaN
9,blocked-shot,73.0,-9.0,NaN
